In [ ]:
from alpaca.data.live import StockDataStream
import pandas as pd
import nest_asyncio
import asyncio

# Inserisci le tue API key Alpaca qui
API_KEY = "AKNKP2ZFQPIEPD7XJBE2"
API_SECRET = "wGeVWSA3lAzp6zHwkdwcbU2w5n9EVC2OZj1B4YWe"
BASE_URL = "https://api.alpaca.markets"

# Lista dei ticker da monitorare
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tic = pd.read_csv(url)
tickers = tic['Symbol'].tolist()

# Crea un DataFrame per accumulare i dati
df_all_data = pd.DataFrame()

# Crea il client per lo streaming
stream = StockDataStream(API_KEY, API_SECRET)

# Callback per gestire i dati ricevuti
async def handle_trade_data(data):
    global df_all_data
    row = {
        "Ticker": data.symbol,
        "Timestamp": pd.to_datetime(data.timestamp, utc=True),
        "Price": data.price,
        "Size": data.size,
        "Exchange": data.exchange
    }
    df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
    print(df_all_data.tail(1))  # Mostra l’ultimo dato ricevuto

# Funzione principale per avviare lo stream
async def main():
    for symbol in tickers[:30]:
        stream.subscribe_trades(handle_trade_data, symbol)
    await stream.run()


nest_asyncio.apply()  # Permette di usare asyncio in un loop già attivo

# Avvia lo stream
await main()

error during websocket communication: connection limit exceeded
Traceback (most recent call last):
  File "/home/elisa-negrini/PycharmProjects/BDT project/.venv/lib/python3.12/site-packages/alpaca/data/live/websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "/home/elisa-negrini/PycharmProjects/BDT project/.venv/lib/python3.12/site-packages/alpaca/data/live/websocket.py", line 135, in _start_ws
    await self._auth()
  File "/home/elisa-negrini/PycharmProjects/BDT project/.venv/lib/python3.12/site-packages/alpaca/data/live/websocket.py", line 126, in _auth
    raise ValueError(msg[0].get("msg", "auth failed"))
ValueError: connection limit exceeded
error during websocket communication: connection limit exceeded
Traceback (most recent call last):
  File "/home/elisa-negrini/PycharmProjects/BDT project/.venv/lib/python3.12/site-packages/alpaca/data/live/websocket.py", line 343, in _run_forever
    await self._start_ws()
  File "/home/elisa-negrini/PycharmProjects/BD

  Ticker                        Timestamp   Price  Size Exchange
0    ACN 2025-04-24 15:45:41.141057+00:00  289.22  10.0        V
  Ticker                        Timestamp  Price   Size Exchange
1    AIG 2025-04-24 15:45:41.697582+00:00  82.31  100.0        V
  Ticker                        Timestamp   Price  Size Exchange
2   ALLE 2025-04-24 15:45:41.890984+00:00  139.03  38.0        V
  Ticker                        Timestamp   Price   Size Exchange
3    ABT 2025-04-24 15:45:42.271793+00:00  129.37  100.0        V
  Ticker                        Timestamp  Price  Size Exchange
4    AOS 2025-04-24 15:45:42.287117+00:00  64.96   8.0        V
  Ticker                        Timestamp  Price   Size Exchange
5    AOS 2025-04-24 15:45:42.300941+00:00  64.97  100.0        V
  Ticker                        Timestamp   Price  Size Exchange
6   ADBE 2025-04-24 15:45:42.323582+00:00  358.93   4.0        V
  Ticker                        Timestamp   Price   Size Exchange
7    ABT 2025-04-24 15:4

In [ ]:
from alpaca.data.live import StockDataStream
import pandas as pd
import nest_asyncio
import asyncio
import os
from datetime import datetime, timedelta

nest_asyncio.apply()

# Configura le API Alpaca
API_KEY = "AKNKP2ZFQPIEPD7XJBE2"
API_SECRET = "wGeVWSA3lAzp6zHwkdwcbU2w5n9EVC2OZj1B4YWe"

# Tickers da monitorare (S&P 500)
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tic = pd.read_csv(url)
tickers = tic['Symbol'].tolist()

# Dati in memoria
df_all_data = pd.DataFrame()

# Stream Alpaca
stream = StockDataStream(API_KEY, API_SECRET)

# Directory per salvataggi
os.makedirs("dati_stream", exist_ok=True)

# Callback per ricevere i trade
async def handle_trade_data(data):
    global df_all_data
    row = {
        "Ticker": data.symbol,
        "Timestamp": pd.to_datetime(data.timestamp, utc=True),
        "Price": data.price,
        "Size": data.size,
        "Exchange": data.exchange
    }
    df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
    print(df_all_data.tail(1))

# Coroutine per salvare i dati periodicamente
async def save_data_periodically(interval_sec=60):
    while True:
        await asyncio.sleep(interval_sec)
        if not df_all_data.empty:
            timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
            path = f"dati_stream/stream_{timestamp}.csv"
            df_all_data.to_csv(path, index=False)
            print(f"💾 Salvato {len(df_all_data)} righe in {path}")

# Funzione principale con tempo limite
async def main(duration_seconds=60):
    for symbol in tickers[:5]:  # Prendiamo solo i primi 5 ticker per evitare eccessivo carico
        stream.subscribe_trades(handle_trade_data, symbol)

    # Avvia salvataggio dati
    save_task = asyncio.create_task(save_data_periodically(interval_sec=30))

    # Tempo di fine
    stop_time = datetime.utcnow() + timedelta(seconds=duration_seconds)

    # Avvia lo stream in parallelo
    stream_task = asyncio.create_task(stream.run())

    # Attendi fino al tempo limite
    while datetime.utcnow() < stop_time:
        await asyncio.sleep(1)

    # Chiudi tutto
    await stream.stop()
    save_task.cancel()
    print("✅ Raccolta dati terminata.")

# Avvio
await main(duration_seconds=60)



/tmp/ipykernel_8183/3123451570.py:60: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stop_time = datetime.utcnow() + timedelta(seconds=duration_seconds)


  Ticker                        Timestamp   Price   Size Exchange
0    MMM 2025-04-24 15:49:36.805213+00:00  138.36  100.0        V
  Ticker                        Timestamp   Price  Size Exchange
1    MMM 2025-04-24 15:49:38.742402+00:00  138.37   2.0        V
  Ticker                        Timestamp   Price  Size Exchange
2    MMM 2025-04-24 15:49:38.742414+00:00  138.37   2.0        V
  Ticker                        Timestamp   Price  Size Exchange
3   ABBV 2025-04-24 15:49:41.603883+00:00  179.77   5.0        V
  Ticker                        Timestamp   Price   Size Exchange
4   ABBV 2025-04-24 15:49:41.603919+00:00  179.77  199.0        V
  Ticker                        Timestamp   Price  Size Exchange
5   ABBV 2025-04-24 15:49:41.714291+00:00  179.78   9.0        V
  Ticker                        Timestamp   Price  Size Exchange
6    ABT 2025-04-24 15:49:55.220559+00:00  129.52   5.0        V


/tmp/ipykernel_8183/3123451570.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


💾 Salvato 7 righe in dati_stream/stream_20250424_154958.csv
  Ticker                        Timestamp   Price  Size Exchange
7    MMM 2025-04-24 15:50:03.481136+00:00  138.37  13.0        V
  Ticker                        Timestamp   Price  Size Exchange
8    MMM 2025-04-24 15:50:03.481145+00:00  138.37   6.0        V
  Ticker                        Timestamp    Price   Size Exchange
9   ABBV 2025-04-24 15:50:06.103539+00:00  179.715  300.0        V
   Ticker                        Timestamp    Price  Size Exchange
10    ABT 2025-04-24 15:50:16.511408+00:00  129.505  50.0        V
   Ticker                        Timestamp    Price  Size Exchange
11    ABT 2025-04-24 15:50:16.511901+00:00  129.505  50.0        V
   Ticker                        Timestamp    Price   Size Exchange
12    ABT 2025-04-24 15:50:19.714083+00:00  129.495  100.0        V
   Ticker                        Timestamp   Price  Size Exchange
13   ABBV 2025-04-24 15:50:25.417576+00:00  179.74  37.0        V
   Ticker 

/tmp/ipykernel_8183/3123451570.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


...

💾 Salvato 43 righe in dati_stream/stream_20250424_155158.csv
   Ticker                        Timestamp   Price   Size Exchange
43   ABBV 2025-04-24 15:51:58.706003+00:00  179.75  100.0        V
   Ticker                        Timestamp   Price   Size Exchange
44    ABT 2025-04-24 15:52:05.122701+00:00  129.46  100.0        V
   Ticker                        Timestamp    Price  Size Exchange
45    MMM 2025-04-24 15:52:05.202479+00:00  138.285  10.0        V
   Ticker                        Timestamp    Price  Size Exchange
46    MMM 2025-04-24 15:52:05.202524+00:00  138.285   8.0        V
   Ticker                        Timestamp    Price  Size Exchange
47    MMM 2025-04-24 15:52:05.395454+00:00  138.285  40.0        V
   Ticker                        Timestamp   Price  Size Exchange
48    ACN 2025-04-24 15:52:05.762720+00:00  289.56  10.0        V
   Ticker                        Timestamp   Price   Size Exchange
49   ABBV 2025-04-24 15:52:10.326866+00:00  179.76  100.0        V
   

/tmp/ipykernel_8183/3123451570.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


💾 Salvato 56 righe in dati_stream/stream_20250424_155228.csv
   Ticker                        Timestamp   Price  Size Exchange
56    ACN 2025-04-24 15:52:28.700573+00:00  289.61   1.0        V
   Ticker                        Timestamp   Price  Size Exchange
57    ACN 2025-04-24 15:52:28.700579+00:00  289.57  50.0        V
   Ticker                        Timestamp   Price  Size Exchange
58    ACN 2025-04-24 15:52:28.700582+00:00  289.56  19.0        V
   Ticker                        Timestamp    Price  Size Exchange
59    ACN 2025-04-24 15:52:28.752718+00:00  289.555   5.0        V
   Ticker                        Timestamp   Price  Size Exchange
60    ACN 2025-04-24 15:52:36.360081+00:00  289.58  10.0        V
   Ticker                        Timestamp    Price  Size Exchange
61    MMM 2025-04-24 15:52:36.454903+00:00  138.325  10.0        V
   Ticker                        Timestamp   Price  Size Exchange
62    ABT 2025-04-24 15:52:36.468025+00:00  129.55  73.0        V
   Ticker  

/tmp/ipykernel_8183/3123451570.py:46: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


💾 Salvato 74 righe in dati_stream/stream_20250424_155258.csv
   Ticker                        Timestamp   Price  Size Exchange
74   ABBV 2025-04-24 15:52:58.373531+00:00  179.79  19.0        V
   Ticker                        Timestamp   Price  Size Exchange
75    ABT 2025-04-24 15:53:03.270476+00:00  129.56  10.0        V
   Ticker                        Timestamp   Price  Size Exchange
76    MMM 2025-04-24 15:53:04.259103+00:00  138.35  10.0        V
   Ticker                        Timestamp    Price  Size Exchange
77    ACN 2025-04-24 15:53:05.022075+00:00  289.825   5.0        V
   Ticker                        Timestamp   Price  Size Exchange
78    ACN 2025-04-24 15:53:06.872563+00:00  289.82  16.0        V


In [ ]:
from alpaca.data.live import StockDataStream
from datetime import datetime, timedelta, timezone
import pytz
import pandas as pd
import nest_asyncio
import asyncio
import os
from datetime import datetime, timedelta

nest_asyncio.apply()

# Configura le API Alpaca
API_KEY = "AKNKP2ZFQPIEPD7XJBE2"
API_SECRET = "wGeVWSA3lAzp6zHwkdwcbU2w5n9EVC2OZj1B4YWe"

# Tickers da monitorare (S&P 500)
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tic = pd.read_csv(url)
tickers = tic['Symbol'].tolist()

# Dati in memoria
df_all_data = pd.DataFrame()

# Stream Alpaca
stream = StockDataStream(API_KEY, API_SECRET)

# Directory per salvataggi
os.makedirs("dati_stream", exist_ok=True)

# Fuso orario Italia
italy_timezone = pytz.timezone("Europe/Rome")

# Callback per ricevere i trade
async def handle_trade_data(data):
    global df_all_data
    row = {
        "Ticker": data.symbol,
        "Timestamp": pd.to_datetime(data.timestamp, utc=True),
        "Price": data.price,
        "Size": data.size,
        "Exchange": data.exchange
    }
    df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
    print(df_all_data.tail(1))

# Coroutine per salvare i dati periodicamente
async def save_data_periodically(interval_sec=60):
    while True:
        await asyncio.sleep(interval_sec)
        if not df_all_data.empty:
            timestamp = datetime.now(italy_timezone).strftime("%Y%m%d_%H%M%S")
            path = f"dati_stream/stream_{timestamp}.csv"
            df_all_data.to_csv(path, index=False)
            print(f"💾 Salvato {len(df_all_data)} righe in {path}")

# Funzione principale con tempo limite
async def main(duration_seconds=60):
    for symbol in tickers[:30]:  # Prendiamo solo i primi 30 ticker perchè non abbiamo il premium di Alpaca
        stream.subscribe_trades(handle_trade_data, symbol)

    # Avvia salvataggio dati
    save_task = asyncio.create_task(save_data_periodically(interval_sec=30))

    # Tempo di fine
    stop_time = datetime.now(italy_timezone) + timedelta(seconds=duration_seconds)

    # Avvia lo stream in parallelo
    stream_task = asyncio.create_task(stream.run())

    # Attendi fino al tempo limite
    while datetime.now(italy_timezone) < stop_time:
        await asyncio.sleep(1)

    # Chiudi tutto
    await stream.stop()
    save_task.cancel()
    print("✅ Raccolta dati terminata.")

# Avvio
await main(duration_seconds=60)



  Ticker                        Timestamp   Price   Size Exchange
0    AES 2025-04-24 15:55:00.280192+00:00  10.285  100.0        V
  Ticker                        Timestamp  Price  Size Exchange
1    LNT 2025-04-24 15:55:00.657872+00:00  61.37   6.0        V
  Ticker                        Timestamp   Price   Size Exchange
2    AEP 2025-04-24 15:55:00.661567+00:00  106.91  100.0        V
  Ticker                        Timestamp   Price  Size Exchange
3   AMZN 2025-04-24 15:55:00.991417+00:00  184.97  16.0        V
  Ticker                        Timestamp   Price  Size Exchange
4   AMZN 2025-04-24 15:55:00.991430+00:00  184.97  84.0        V
  Ticker                        Timestamp  Price  Size Exchange
5    AEE 2025-04-24 15:55:01.086982+00:00  99.16  40.0        V
  Ticker                        Timestamp  Price  Size Exchange
6    AEE 2025-04-24 15:55:01.087348+00:00  99.16  40.0        V
  Ticker                        Timestamp  Price  Size Exchange
7    AEE 2025-04-24 15:55:01

In [ ]:
from alpaca.data.live import StockDataStream
from datetime import datetime, timedelta
import pytz
import pandas as pd
import nest_asyncio
import asyncio
import os

# Applica patch per notebook o ambienti embedded (opzionale in PyCharm)
nest_asyncio.apply()

# Configura le API Alpaca
API_KEY = "AKNKP2ZFQPIEPD7XJBE2"
API_SECRET = "wGeVWSA3lAzp6zHwkdwcbU2w5n9EVC2OZj1B4YWe"

# Legge tickers S&P500
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tic = pd.read_csv(url)
tickers = tic['Symbol'].tolist()

# Dati raccolti
df_all_data = pd.DataFrame()

# Stream Alpaca
stream = StockDataStream(API_KEY, API_SECRET)

# Directory per salvataggi
os.makedirs("dati_stream", exist_ok=True)

# Fuso orario
italy_timezone = pytz.timezone("Europe/Rome")

# Ricezione trade
async def handle_trade_data(data):
    global df_all_data
    row = {
        "Ticker": data.symbol,
        "Timestamp": pd.to_datetime(data.timestamp, utc=True),
        "Price": data.price,
        "Size": data.size,
        "Exchange": data.exchange
    }
    df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
    print(df_all_data.tail(1))

# Salvataggio periodico
async def save_data_periodically(interval_sec=30):
    while True:
        await asyncio.sleep(interval_sec)
        if not df_all_data.empty:
            timestamp = datetime.now(italy_timezone).strftime("%Y%m%d_%H%M%S")
            path = f"dati_stream/stream_{timestamp}.csv"
            df_all_data.to_csv(path, index=False)
            print(f"💾 Salvato {len(df_all_data)} righe in {os.path.abspath(path)}")

# Funzione principale
async def main(duration_seconds=60):
    global df_all_data

    # Abbonati ai primi 30 ticker (Alpaca free limit)
    for symbol in tickers[:30]:
        stream.subscribe_trades(handle_trade_data, symbol)

    # Task salvataggio
    save_task = asyncio.create_task(save_data_periodically(interval_sec=20))

    # Stop dopo X secondi
    stop_time = datetime.now(italy_timezone) + timedelta(seconds=duration_seconds)

    # Avvia streaming
    stream_task = asyncio.create_task(stream.run())

    # Attesa
    while datetime.now(italy_timezone) < stop_time:
        await asyncio.sleep(1)

    # Stop streaming
    await stream.stop()

    # Ferma salvataggio periodico
    save_task.cancel()

    # Salvataggio finale
    if not df_all_data.empty:
        final_timestamp = datetime.now(italy_timezone).strftime("%Y%m%d_%H%M%S")
        final_path = f"dati_stream/stream_{final_timestamp}_FINAL.csv"
        df_all_data.to_csv(final_path, index=False)
        print(f"✅ Salvataggio finale con {len(df_all_data)} righe in {os.path.abspath(final_path)}")

    print("✅ Raccolta dati terminata.")

# Avvio dello script (solo in esecuzione diretta)
if __name__ == "__main__":
    asyncio.run(main(duration_seconds=60))


  Ticker                        Timestamp   Price  Size Exchange
0    ARE 2025-04-24 16:08:26.092542+00:00  77.265   2.0        V
  Ticker                        Timestamp   Price  Size Exchange
1    AXP 2025-04-24 16:08:26.138338+00:00  265.19  45.0        V
  Ticker                        Timestamp   Price  Size Exchange
2    AXP 2025-04-24 16:08:26.138617+00:00  265.19  25.0        V
  Ticker                        Timestamp  Price  Size Exchange
3    ARE 2025-04-24 16:08:26.159340+00:00  77.27   1.0        V
  Ticker                        Timestamp  Price  Size Exchange
4   AKAM 2025-04-24 16:08:26.194472+00:00  77.92  16.0        V
  Ticker                        Timestamp   Price  Size Exchange
5   ABNB 2025-04-24 16:08:26.221289+00:00  120.39  50.0        V
  Ticker                        Timestamp  Price   Size Exchange
6   ABNB 2025-04-24 16:08:26.221627+00:00  120.4  100.0        V
  Ticker                        Timestamp  Price  Size Exchange
7   ABNB 2025-04-24 16:08:26.2